In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
161,AFG,Asia,Afghanistan,2020-06-19,27532.0,658.0,546.0,42.0,707.248,16.903,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
264,ALB,Europe,Albania,2020-06-19,1788.0,66.0,39.0,1.0,621.308,22.934,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
431,DZA,Africa,Algeria,2020-06-19,11385.0,117.0,811.0,12.0,259.629,2.668,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-06-18,France,158548.0,0.0,29668.0,0.0,0.0,0.0
0,2020-06-18,China,84534.0,0.0,4650.0,0.0,0.0,0.0
0,2020-06-18,Italy,238173.0,0.0,34505.0,0.0,0.0,0.0
0,2020-06-18,Spain,244970.0,0.0,29069.0,0.0,0.0,0.0
0,2020-06-18,United States,2189206.0,0.0,118678.0,0.0,0.0,0.0
0,2020-06-18,World,8468802.0,0.0,457494.0,0.0,0.0,0.0
0,2020-06-18,United Kingdom,300899.0,0.0,42422.0,0.0,0.0,0.0
0,2020-06-18,Germany,188184.0,0.0,8893.0,0.0,0.0,0.0
0,2020-06-18,Iran,197845.0,0.0,9294.0,0.0,0.0,0.0
0,2020-06-18,Turkey,184088.0,0.0,4877.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-06-19,France,158548.0,158641.0,29668.0,29603.0,-93.0,65.0
0,2020-06-19,China,84534.0,84494.0,4650.0,4638.0,40.0,12.0
0,2020-06-19,Italy,238173.0,238159.0,34505.0,34514.0,14.0,-9.0
0,2020-06-18,Spain,244970.0,245268.0,29069.0,29054.0,-298.0,15.0
0,2020-06-19,United States,2189206.0,2191052.0,118678.0,118434.0,-1846.0,244.0
0,2020-06-19,World,8468802.0,8455738.0,457494.0,455761.0,13064.0,1733.0
0,2020-06-19,United Kingdom,300899.0,300469.0,42422.0,42288.0,430.0,134.0
0,2020-06-19,Germany,188184.0,187764.0,8893.0,8856.0,420.0,37.0
0,2020-06-19,Iran,197845.0,197647.0,9294.0,9272.0,198.0,22.0
0,2020-06-19,Turkey,184088.0,184031.0,4877.0,4882.0,57.0,-5.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")